In [13]:
import subprocess
import requests
import xml.etree.ElementTree as ET
import openpyxl
from openpyxl.styles import Alignment, Font, PatternFill
from openpyxl.drawing.image import Image as ExcelImage
from io import BytesIO

def px_to_row_height(px: int) -> float :
    return px * 0.75
    
def px_to_col_width(px: int) -> float :
    return (px - 14) / 7
    

BASE_URL = "https://apis.data.go.kr/B551011/GoCamping/basedList"
API_KEY = "bb54fc1f25d5b537cfbc5d1430aa31b14cc8f65a0710615096749046f79033e7"

url = f"{BASE_URL}?numOfRows=50&pageNo=1&MobileOS=ETC&MobileApp=camping&serviceKey={API_KEY}"

result = subprocess.run(["curl", "-k", url], capture_output=True, text=True, encoding="utf-8")
root = ET.fromstring(result.stdout)

wb= openpyxl.Workbook()
ws = wb.active
ws.title = "캠핑장정보"

headers = ["아이디", "캠핑장명", "이미지", "지역", "주소" , "홈페이지"]
ws.append(headers)

IMG_W_PX = 100
IMG_H_PX = 75
ROW_H_PT = px_to_row_height(IMG_H_PX)
COL_W_PT = px_to_col_width(IMG_W_PX)

cell_align = Alignment(vertical="center", horizontal="left", wrap_text=True)

header_fill = PatternFill("solid", fgColor="F2F2F2")
header_font = Font(bold=True)
for col in range(1,len(headers) + 1) :
    c = ws.cell(row=1, column=col)
    c.alignment = Alignment(vertical="center", horizontal="center")
    c.fill = header_fill
    c.font = header_font

items = root.findall(".//item")
row = 2

for item in items : 
    contentId = item.findtext("contentId", default="")
    facltNm = item.findtext("facltNm", default="")
    firstImageUrl = item.findtext("firstImageUrl", default="")
    doNm = item.findtext("doNm", default="")
    addr1 = item.findtext("addr1", default="")
    homepage = item.findtext("homepage", default="")

    ws.append([contentId, facltNm, "", doNm, addr1, homepage])
    
    ws.row_dimensions[row].height = ROW_H_PT
    
    for col in range(1, 7) :
        ws.cell(row=row, column=col).alignment = cell_align

    if firstImageUrl :
        try : 
            img_data = requests.get(firstImageUrl, timeout=10).content
            img_file = BytesIO(img_data)
            img = ExcelImage(img_file)
            img.width = IMG_W_PX
            img.height = IMG_H_PX
            ws.add_image(img, f"C{row}")
        except Exception as e :
            print(f"[경고] 이미지 다운로드 실패 : {firstImageUrl} -> e")
            
    row += 1

ws.column_dimensions["A"].width = 12
ws.column_dimensions["B"].width = 30
ws.column_dimensions["C"].width = COL_W_PT
ws.column_dimensions["D"].width = 15
ws.column_dimensions["E"].width = 50
ws.column_dimensions["F"].width = 50

file_name = "camping_info.xlsx"

ws.freeze_panes = "A2"

wb.save(file_name)
print(f"✅{file_name} 파일저장 완료")

✅camping_info.xlsx 파일저장 완료
